# Data preprocesing  and cleaning

- In this notebook we post process the gathered product data from the scrapy spider
- And find the relevant products using NLP or rules
- Reference - https://github.com/jithinanievarghese/product-search-relevance/

In [1]:
from string import punctuation
from nltk.tokenize import word_tokenize
from rapidfuzz import fuzz
from csv import QUOTE_ALL
from time import time
import pandas as pd


In [2]:

def sample(df):
    """
    return sample view of dataframe
    """
    BOLD = '\033[1m'
    END = '\033[0m'
    sample_df = df.sample(1)
    for row , col in zip(sample_df.values[0], sample_df.columns):
        print(f'{BOLD}{col}{END}\n\n{row}\n')


# Read Data

In [3]:
df = pd.read_json("outputs/listing_data.jsonl", lines=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  2460 non-null   object
 1   subtitle               2328 non-null   object
 2   product_url            2460 non-null   object
 3   smart_url              2460 non-null   object
 4   product_id             2460 non-null   object
 5   image_url              2460 non-null   object
 6   total_count            2460 non-null   int64 
 7   next_listing_page_url  2399 non-null   object
 8   search_query           2460 non-null   object
 9   page                   2460 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 192.3+ KB


In [5]:
sample(df)

title

Aapaga Magic 2x2 High Speed Cube Puzzle Toy for Kids & Adults | 2x2x2 Stickerless cubes

subtitle

1 Pieces

product_url

https://www.flipkart.com/aapaga-magic-2x2-high-speed-cube-puzzle-toy-kids-adults-2x2x2-stickerless-cubes/p/itmdd0a15ea444d2?pid=PUZGAH8VZZQBQQ4B

smart_url

http://dl.flipkart.com/dl/aapaga-magic-2x2-high-speed-cube-puzzle-toy-kids-adults-2x2x2-stickerless-cubes/p/itmdd0a15ea444d2?pid=PUZGAH8VZZQBQQ4B

product_id

PUZGAH8VZZQBQQ4B

image_url

http://rukmini1.flixcart.com/image/612/612/kybvo280/puzzle/k/h/g/1-magic-2x2-high-speed-cube-puzzle-toy-for-kids-adults-2x2x2-original-imagah8vuem6tfg5.jpeg?q=70

total_count

322

next_listing_page_url

https://www.flipkart.com/search?q=spider+man+car+toy&page=4

search_query

spider man car toy

page

3



In [6]:
df.drop_duplicates(subset=["product_id"], inplace=True)

In [7]:
df.title.unique()

array(['RVM Toys Spiderman Bobble Head Action Figure Toy Showpiece Bobblehead For Car / Office',
       'Aapaga Action Figure Super Heros Toy Set | Inspired By Avengers Marvel Characters Iron Man, Hulk, Thor, Captain America And Ant Man Toys Collection For Kids | Multi-Color | Size: 4.5 Inches | Set Of 5',
       'Daiyamondo Red Spider Spring Head Moving Had Bobble head Spring Dancing PVC Bobble Spring Dancing Doll Toy Car Dashboard Bounce Toys for Car Interior Dashboard Expression BobbleHead (Multicolor)',
       ...,
       'SUBH-ARAMBH SUPER HERO (SPIDER MAN) UNBREAKABLE FRICTION POWERED TOY CAR WITH MOVING HEAD',
       'PITARA Spider Men Pressure Powered Press The Head Kids Toddler Car Toy',
       'Aseenaa Angry Bird Study Game Toy laptop With Music And Alphabet Sound And Lights for new Kids | Educational mini Laptops | Learning small computer gaming Toys | Best Gift For boys girls Toddler kid | Color Multi'],
      dtype=object)

In [8]:
df.shape

(2233, 10)

# Image Data Saved or Upload Status
 - Here we used Deta Cloud Drive (https://www.deta.sh/)  for uploading the images of products while scraping
 - We can also save the data in local if there is no Deta project key
 - we also checked whether there is any missing in the product image data, by comparing with the product details dataset
 - some of the products had duplicated images so the there is less count in image datasets


In [9]:

upload_status = pd.read_json("outputs/upload_status.jsonl", lines=True)

In [10]:
upload_status.shape

(1697, 4)

In [11]:
set(df.image_url) - set(upload_status.image_url)

set()

# Data Cleaning and string matching

- We can either use string_match_with_rules() or string_match_with_fuzz(). 

### string_match_with_rules()
- Uses a rule searching simple approach
- With help of some known or expected keywords like "spiderman", "spider man" , "spidey" , "spider" in product title
- Fast compared to string_match_with_fuzz().

### string_match_with_fuzz() 

- Uses [Levenshtein Distance](https://medium.com/analytics-vidhya/fuzzy-matching-in-python-2def168dee4a) string matching algorithm.
- we first tokenize the text
- then take the partial ratio of each text with one keyword `spider`
- if any of the text have a match greater than 90 then it will be a match

- if we are using string_match_with_rules(), then we have to add more domain specific keywords in our rule search, 
- But for string_match_with_fuzz() we only had to add a keyword "spider". 

- So if our problem statement have more search keywords, it is always better to use Levenshtein Distance to calculate the match




In [13]:
from string import punctuation
from nltk.tokenize import word_tokenize
from rapidfuzz import fuzz

def clean_white_space(text):
    """
    to clean unwanted white space in text
    """
    if not text:
        return
    return " ".join(text.split())

def process_title(text):
    """
    to clean the text by lowercasing,
    and removing special characters and digits
    """
    text = text.lower()
    text = "".join([char for char in text if char not in punctuation and not char.isdigit()])
    return clean_white_space(text)

def string_match_with_fuzz(text, threshold=90):
    """
    to return true if text have a match with fuzz partial ratio
    """
    text = process_title(text)
    # tokenize the text using nltk tokenizer
    text_list = word_tokenize(text)
    ratios = [fuzz.partial_ratio(text_, "spider") if len(text_) > 3 else 0 for text_ in text_list]
    if any([ratio_ >= threshold for ratio_ in ratios]):
        return True
    

def string_match_with_rules(text):
    """
    to return True if the following keywords are present
    in the text
    """
    text = process_title(text)
    naive_search = ["spiderman" in text, "spider man" in text, "spidey" in text, "spider" in text]
    if any(naive_search):
        return True
    


In [14]:
title = "SEMAPHORE bobblehead Toys Action Figure and Car Dashboard Interior Accessories(SPIDERMAN) Compatible with Hyundai Verna"
print("product title:",title)
start = time()
print("\nstring match with rules , title match:", string_match_with_rules(title))
print(f"time taken to process {time()-start} seconds\n")
start = time()
print("string match with Levenshtein Distance (partial ratio), title match:", string_match_with_fuzz(title))
print(f"time taken to process {time()-start} seconds")

product title: SEMAPHORE bobblehead Toys Action Figure and Car Dashboard Interior Accessories(SPIDERMAN) Compatible with Hyundai Verna

string match with rules , title match: True
time taken to process 8.940696716308594e-05 seconds

string match with Levenshtein Distance (partial ratio), title match: True
time taken to process 0.004745006561279297 seconds


In [15]:
title = "Men Full Sleeve Printed Hooded Sweatshirt"
print("product title:",title)
start = time()
print("\nstring match with rules , title match:", string_match_with_rules(title))
print(f"time taken to process {time()-start} seconds\n")
start = time()
print("string match with Levenshtein Distance (partial ratio), title match:", string_match_with_fuzz(title))
print(f"time taken to process {time()-start} seconds")

product title: Men Full Sleeve Printed Hooded Sweatshirt

string match with rules , title match: None
time taken to process 0.00017952919006347656 seconds

string match with Levenshtein Distance (partial ratio), title match: None
time taken to process 0.0004837512969970703 seconds


#### Why fuzz.partial ratio()
- For our use case, partial ratio perfomed well in identifying product title, at the same time avoiding unwanted titles matching (when we kept a threshold of 90).

- we can also combine more than one ratios score and then match strings


In [12]:
test_texts = ["spidey", "spider", "spiderman", "spider-man", "ironman", "super", "sticker", 'spin', 'd']
result = []
search_key = 'spider'
for test_text in test_texts:
    result.append({"keyword": test_text,
    "fuzz.ratio": fuzz.ratio(search_key, test_text), 
    "fuzz.partial_ratio": fuzz.partial_ratio(search_key, test_text),
    "fuzz.token_sort": fuzz.token_sort_ratio(search_key, test_text),
    "fuzz.token_set": fuzz.token_set_ratio(search_key, test_text)
    })
pd.DataFrame(result)

,keyword,fuzz.ratio,fuzz.partial_ratio,fuzz.token_sort,fuzz.token_set
0,spidey,83.333333,90.909091,83.333333,83.333333
1,spider,100.000000,100.000000,100.000000,100.000000
2,spiderman,80.000000,100.000000,80.000000,80.000000
3,spider-man,75.000000,100.000000,75.000000,100.000000
4,ironman,30.769231,50.000000,30.769231,30.769231
5,super,72.727273,60.000000,72.727273,72.727273
6,sticker,61.538462,54.545455,61.538462,61.538462
7,spin,60.000000,85.714286,60.000000,60.000000
8,d,28.571429,100.000000,28.571429,28.571429


# String matching Speed comparison, rules vs fuzz on entire datasets

In [16]:
%%time
df["relevant_products_string_match"] = df.title.apply(lambda x: string_match_with_rules(x))

CPU times: user 16.7 ms, sys: 740 µs, total: 17.5 ms
Wall time: 17.3 ms


In [17]:
%%time
df["relevant_products_fuzz_match"] = df.title.apply(lambda x: string_match_with_fuzz(x))

CPU times: user 197 ms, sys: 3.29 ms, total: 200 ms
Wall time: 200 ms


In [18]:

unidentified_products = df[df.relevant_products_string_match.isnull()]
unidentified_products.shape

(1058, 12)

In [19]:
relevant_products = df[~df.relevant_products_fuzz_match.isnull()]
relevant_products.shape

(1175, 12)

 total 1175 products were identified as relevant products from product title

# Remove image url duplicates

In [20]:
relevant_products = relevant_products.drop_duplicates(subset=['image_url'])
relevant_products.shape

(1124, 12)

In [21]:
relevant_products = relevant_products.drop_duplicates(subset=['image_url'])

In [22]:
relevant_products.columns

Index(['title', 'subtitle', 'product_url', 'smart_url', 'product_id',
       'image_url', 'total_count', 'next_listing_page_url', 'search_query',
       'page', 'relevant_products_string_match',
       'relevant_products_fuzz_match'],
      dtype='object')

In [23]:
final_data = relevant_products.loc[:, ["product_id", "title", "product_url",  'image_url']]

In [24]:
final_data.shape

(1124, 4)

In [25]:
final_data.to_csv("relevant_products.csv", index=False, quoting=QUOTE_ALL)